In [ ]:

######========================================================================######

# Cody:
# How to parse into the data frame to drop certain rows:
# check with Joseph for more info on this

t1_nan_z2016 = nan_z2016.drop(nan_z2016[
    (nan_z2016['calculatedfinishedsquarefeet']<= 400)
].index)

t2_nan_z2016 = t1_nan_z2016.drop(t1_nan_z2016[
    (t1_nan_z2016['bedroomcnt']<= 0)
].index)

t3_nan_z2016 = t2_nan_z2016.drop(t2_nan_z2016[
    (t2_nan_z2016['bathroomcnt']<= 0)
].index)

t4_nan_z2016 = t3_nan_z2016.drop(t3_nan_z2016[
    (t3_nan_z2016['lotsizesquarefeet']<=
     t3_nan_z2016['calculatedfinishedsquarefeet'])].index)


parsed_data = t4_nan_z2016.sort_values(by=['lotsizesquarefeet',
    'bedroomcnt',
    'bathroomcnt',
    'calculatedfinishedsquarefeet'], ascending=True)

df = df.drop(df[df['field'] <= 0)].index)
# how to parse into a column into a dataframe and count instances of the values in that column
parsed_data.groupby('bedroomcnt').count()[['parcelid']]


######========================================================================######
# Norrick:
# get a copy of his initial functins to check the data, nans, counts, etc.
# copy his feature selection and imputing

def percent_zeros(df):
    cols = list(df.columns)
    zeros = {}
    for col in cols:
        zeros[col] = 'Zeros: {:.2f}%'.format((df[col] == 0).sum()/df.shape[0] * 100)
        
    return zeros

def percent_NaNs(df):
    cols = list(df.columns)
    Nans = {}
    for col in cols:
        Nans[col] = 'Missing {:0.2f}%'.format(df[col].isnull().sum()/df.shape[0]*100)
        
    return Nans

def replace_values(df,entry,orginal_value,new):
    df[entry] = df[entry].replace(orginal_value,new)
    return df

def drop_nulls_from(df,column):
    return df[df[column].notnull()]


Let's see if we can capture info on what is driving the error with regression models

Feature Selection and Imputing

df = drop_nulls_from(df, 'taxamount')
df = drop_nulls_from(df, 'taxvaluedollarcnt' )
df = replace_values(df,'garagecarcnt',np.nan,0)
df = replace_values(df,'poolcnt',np.nan,0)
df = replace_values(df,'yearbuilt',np.nan, df['yearbuilt'].mean())
df = replace_values(df,'garagecarcnt',np.nan,0)
df = replace_values(df,'calculatedfinishedsquarefeet',np.nan,0)
df = replace_values(df,'fullbathcnt',np.nan,0)

######========================================================================######

# me

######========================================================================######
# Katy:
Explore the data more at the offset, like Maggie did, very methodically and applied life-knowledge and industry-knowledge

######========================================================================######

# Stephen:
grouping, 4-line code he hated... looked good    
.duplicated (finds dupes)
warning flag for dups
like his logic on fillna (how=any, ...) copy this
df = np.log(df.field)
get his logic on dropping rows... i like this.

# Jason:

# Gary:
# dupe logic and one-to-one logic
pd.concat(dupes for _, dupes in logerrors.groupby("parcelid") if len(dupes) > 1)

#Now remove the duplicates (keeping the last occurance) based on parcel id
logerrors.drop_duplicates(subset='parcelid', keep='last', inplace=True)

# and merge the two dataframes, checking for duplicates
frames = [properties, logerrors]
df = pd.merge(properties, logerrors, on='parcelid', how='inner', validate="one_to_one")
df.head()






# Joseph:
I like his data prep stage and removing this and that... get this logic.
I like this also:   df.isnull().sum    ...use this more
copy both of these


# Orion:


######========================================================================######

# Michael:
functions, many and handy
used our logic to get rid of outlier
remove outlier:  can also use the logic of percentiles, ie drop above 95% and below 5%

def df_print_r_and_p_values(X, y):
    r_and_p_values = {col: stats.pearsonr(X[col], y) for col in X.columns}
    print("PEARSON'S R")
    for k, v in r_and_p_values.items():
        col = k
        r, p = v
        print(f"{col}:")
        print(
            f"\tPearson's R is {r:.2f} with a significance p-value of {p: .3}\n"
        )


def linreg_fit_and_predict(x_train, y_train, x_test, y_test):
    lm = LinearRegression()
    lm.fit(x_train, y_train)

    y_label = y_train.columns[0]
    y_intercept = lm.intercept_[0]
    m = lm.coef_[0][0]
    x_label = x_train.columns[0]
    print(f"Univariate: {y_label} = {y_intercept:.2f} + {m:.3}*{x_label}")
    print()

    preds_train = lm.predict(x_train)

    # run test data through model
    preds_test = lm.predict(x_test)

    return lm, preds_train, preds_test


def evaluate_model_train(x, y, preds):
    y_label = y.columns[0]
    x_label = x.columns[0]

    print("Model Evaluation on TRAIN Data")
    meanse = mean_squared_error(y, preds)
    print(f"\tMSE: {meanse:.3f}")

    medianae = median_absolute_error(y, preds)
    print(f"\tMAE: {medianae:.3f}")

    r2 = r2_score(y, preds)
    print(
        f"\t{r2:.2%} of the variance in {y_label} can be explained by {x_label}."
    )
    print()

    print("P-VALUE")
    f_vals, p_vals = f_regression(x, y)
    print(f"\tTrain: {p_vals[0]:.3}")
    print()


def evaluate_model_test(x, y, preds):
    y_label = y.columns[0]
    x_label = x.columns[0]

    print("Model Evaluation on TEST Data")
    meanse = mean_squared_error(y, preds)
    print(f"\tMSE: {meanse:.3f}")

    medianae = median_absolute_error(y, preds)
    print(f"\tMAE: {medianae:.3f}")

    r2 = r2_score(y, preds)
    print(
        f"\t{r2:.2%} of the variance in {y_label} can be explained by {x_label}."
    )
    print()

    print("P-VALUE")
    f_vals, p_vals = f_regression(x, y)
    print(f"\tTest: {p_vals[0]:.3}")
    print()


def plot_residuals(y_test, preds_test):
    y_label = y_test.columns[0]
    plt.scatter(preds_test, preds_test - y_test, c="g", s=20)
    plt.hlines(y=0, xmin=preds_test.min(), xmax=preds_test.max())
    plt.title("Residual plot")
    plt.ylabel("Residuals")
    plt.xlabel(y_label)
    plt.show()


def linreg_model(x_train, y_train, x_test, y_test):
    lm, preds_train, preds_test = linreg_fit_and_predict(
        x_train, y_train, x_test, y_test
    )

    evaluate_model_train(x_train, y_train, preds_train)
    evaluate_model_test(x_test, y_test, preds_test)

    plot_residuals(y_test, preds_test)
    

def evaluate_multi_model_train(X, y, preds):
    y_label = y.columns[0]
    X_labels = X.columns

    print("Model Evaluation on TRAIN Data")
    meanse = mean_squared_error(y, preds)
    print(f"\tMSE: {meanse:.3f}")

    medianae = median_absolute_error(y, preds)
    print(f"\tMAE: {medianae:.3f}")

    r2 = r2_score(y, preds)
    print(
        f"\t{r2:.2%} of the variance in {y_label} can be explained by {X_labels}."
    )
    print()

    print("P-VALUE")
    f_vals, p_vals = f_regression(X, y)
    print(f"\tTrain: {p_vals[0]:.3}")
    print()
    
def evaluate_multi_model_test(X, y, preds):
    y_label = y.columns[0]
    X_labels = X.columns

    print("Model Evaluation on TEST Data")
    meanse = mean_squared_error(y, preds)
    print(f"\tMSE: {meanse:.3f}")

    medianae = median_absolute_error(y, preds)
    print(f"\tMAE: {medianae:.3f}")

    r2 = r2_score(y, preds)
    print(
        f"\t{r2:.2%} of the variance in {y_label} can be explained by {X_labels}."
    )
    print()

    print("P-VALUE")
    f_vals, p_vals = f_regression(X, y)
    print(f"\tTest: {p_vals[0]:.3}")
    print()
    
def multi_linreg_fit_and_evaluate(X_train, y_train, X_test, y_test):
    lm = LinearRegression()
    lm.fit(X_train, y_train)

    y_label = y_train.columns[0]
    y_intercept = lm.intercept_[0]
    print("Multivariate:")
    print(f"{y_label} = ")
    print(f"{y_intercept:.3f}")
    for i, col in enumerate(X_train.columns):
        coefficient = lm.coef_[0][i]
        print(f"+ {coefficient:.3}*{col}")

    preds_train = lm.predict(X_train)
    evaluate_multi_model_train(X_train, y_train, preds_train)
    
    preds_test = lm.predict(X_test)
    evaluate_model_test(X_test, y_test, preds_test)
    
    plot_residuals(y_test, preds_test)


def normalize_cols(df_train, df_test, cols):
    df_train_norm = pd.DataFrame()
    for col in cols:
        minimum = df_train[col].min()
        maximum = df_train[col].max()
        df_train_norm[f"{col}_norm"] = (df_train[col] - minimum) / (maximum - minimum)
    
    df_test_norm = pd.DataFrame()
    for col in cols:
        minimum = df_train[col].min()  # use the min and max from the train set
        maximum = df_train[col].max()
        df_test_norm[f"{col}_norm"] = (df_test[col] - minimum) / (maximum - minimum)
    return df_train_norm, df_test_norm    
    
######========================================================================######    
    
    
    
    
# Nicole:

# Matt:
Handy:  df.field.valuecounts()
plt.subplot logic in cell 56
sns.relplot.group by and count year in cell 124

# Matthew:
cell 32 and 33, binning, grouping, etc.

# Eric
cell 130 replace nulls



def percent_missing(df):
	missing_table = df.isnull().sum()/df.shape[0]*100
	return missing_table

df['bedbath'] = df['bedroomcnt'] + df['bathroomcnt']
df['log_abs'] = df['logerror']
df['roombedbath_ratio'] = df['bedbath']/df['roomcnt']
df['sqft_per_room'] = df['calculatedfinishedsquarefeet']/df['roomcnt']
df['dollar_per_sqft'] = df['taxvaluedollarcnt']/df['calculatedfinishedsquarefeet']



